# LDA and Document Similarity 
We are using Gensim's LDA model and diff method.

## Loading all the libraries in

In [31]:
# Importing all lib 
import re
import numpy as np
import pandas as pd
from pprint import pprint

from nltk import FreqDist
from scipy.stats import entropy

## Gensim
from gensim import corpora, models
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel
from gensim.test.utils import common_corpus, common_dictionary

##SKLearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


## spacy for lemmatization
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load()

## Plotting tools
import pyLDAvis
import pyLDAvis.gensim  ## don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

## Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Preprocessing
## First part of the preprocessing will be importing the Data set from GitHub, and loading it into a dataframe.

In [32]:
# Import Dataset
#df = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2016.json')
df1 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2015.json')
df2 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2014.json')
df3 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2013.json')
df4 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2012.json')
df5 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2011.json')
df6 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2010.json')
df7 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2009.json')
df8 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2008.json')
df9 = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2006.json')

print(df1.abstracts.unique())
df.head()

['A formal qualitative risk management approach for IT security Information technology (IT) security, which is concerned about protecting the confidentiality, integrity and availability of information technology assets, inherently possesses a significant amount of risk, some known and some unknown. IT security risk management has gained considerable attention over the past decade due to the collapsing of some large organisations in the world. Previous investigative research in the field of IT security have indicated that despite the efforts that organisations employ to reduce IT security risks, the trend of IT security attacks are still increasing. One of the contributing factors to poor management of IT security risk is attributed to the fact that IT security risk management is often left to the technical security technologist who do not necessarily employ formal risk management tools and reasoning. For this reason, organisations find themselves in a position where they do not have th

,abstracts,title
0,Team Formation in Digital Forensics A major pr...,Team Formation in Digital Forensics


### Removing unwanted characters
\begin{itemize}
\item Convert to list
\item Remove Email prefixes
\item Remove New lines
\item Replace Ligature characters
\item Remove single quotes
\end{itemize}



In [33]:
#data1 = df.abstracts.values.tolist()
data2 = df1.abstracts.values.tolist()
data3 = df2.abstracts.values.tolist()
data4 = df3.abstracts.values.tolist()
data5 = df4.abstracts.values.tolist()
data6 = df5.abstracts.values.tolist()
data7 = df6.abstracts.values.tolist()
data8 = df7.abstracts.values.tolist()
data9 = df8.abstracts.values.tolist()
data10 = df9.abstracts.values.tolist()

data = list(set(data2 + data3 + data4+ data5+ data6+ data7+ data8+ data9+ data10))

## Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

## Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

## Remove fi ligature characters
data = [re.sub('ﬁ', 'fi', sent) for sent in data]

## Remove fi ligature characters
data = [re.sub('ﬂ', 'fl', sent) for sent in data]

## Remove single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[1:])

['Guidelines for Procedures of a Harmonised Digital Forensic Process in '
 'Network Forensics Cloud computing is a new computing paradigm that presents '
 'fresh research issues in the field of digital forensics. Cloud computing '
 'builds upon virtualisation technologies and is distributed in nature. '
 'Depending on its implementation, the cloud can span across numerous '
 'countries. Its distributed nature and virtualisation introduces digital '
 'forensic research issues that include among others difficulty in identifying '
 'and collecting forensically sound evidence. Even if the evidence may be '
 'identified and essential tools for collecting the evidence are acquired, it '
 'may be illegal to access computer data residing beyond the jurisdiction of a '
 'forensic investigator. The investigator needs to acquire a search warrant '
 'that can be executed in a specific foreign country – which may not be a '
 'single country due to the distributed nature of the cloud. Obtaining '
 '

 'still find it a challenge to comply with data privacy laws that have been '
 'enacted there, a while ago. This article proposes a methodology to comply '
 'with POPI. The Generally Accepted Privacy Principles (GAPP) is an '
 'American/Canadian framework containing international privacy requirements '
 'with best practices. Both, POPI and GAPP address a common purpose: ‘How '
 'personal information is collected, used, retained, disclosed, and disposed.’ '
 'GAPP is reputed as a solid benchmark for good privacy practice, comprising '
 'of ten overarching privacy principles which yields a set of criteria for '
 'effective management of privacy risks and compliance. Much of the provisions '
 'in POPI is addressed in GAPP. A key condition (Security Safeguards) in POPI '
 'stipulates what aspects of personal information must be adequately secured, '
 'with limited insight on how to go about this process. Accordingly, this '
 'article proposes a methodology to fill this gap. All of the prov

 'specifications are automatically generated from configuration files and '
 'component codes. Test cases are then automatically generated from '
 'vulnerability patterns and the previous specifications. A tool, called '
 'APSET, is presented and evaluated with experimentations on some Android '
 'applications.',
 'A SURVEY OF COMPUTER CRIME AND SECURITY IN SOUTH AFRICA ',
 'SMT-Constrained Symbolic Execution Engine for Integer Overflow Detection in '
 'C Code A sentiment that is quite often encountered in database forensic '
 'research material is the scarcity of scientific research in this vital area '
 'of digital forensics. Databases have been around for many years in the '
 'digital space and have moved from being exclusively used in specialised '
 'applications of big corporations to becoming a means to an end in even the '
 'simplest end-user applications. Newer disciplines such as cloud forensics '
 'seem to be producing a far greater volume of new research material than '
 'da

 'and its foreign intelligence partners known as the Five Eyes [1]. The '
 'documents revealed secret programs about the NSAs mass bulk collection of '
 'phone, internet and communications traffic as well as how the NSA and its '
 'partners are working to sabotage and weaken encryption algorithms and the '
 'security protocols used to secure the internet. This paper presents some of '
 'the programs that were revealed as well as the rationale and legislation '
 'behind these programs from a global perspective. Mass surveillance is not '
 'only done by the Five Eyes partners but also by many other countries who pay '
 'private companies to provide them with tools to spy, censor and repress '
 'their own citizens [2]. In order to assess the potential harm and the '
 'security implications of mass surveillance, this paper looks at how state '
 'level actors around the world are conducting surveillance which raise '
 'broader issues about internet security such as how common weaknesses are

 'as well as the number of traversals through the directory structure and are '
 'used throughout much of the research in the paradigm of lightweight '
 'classification. Methods which include external sources of information are '
 'often called fully featured classifications and have been shown to be only '
 'slightly more effective than a purely lexical analysis when considering both '
 'false-positives and false-negatives. This distinction allows these '
 'algorithms to be run client side without the introduction of additional '
 'latency, but still providing a high level of accuracy through the use of '
 'modern techniques in training classifiers. Analysis of this type will also '
 'be useful in an incident response analysis where large numbers of URLs need '
 'to be filtered for potentially malicious URLs as an initial step in '
 'information gathering as well as end user implementations such as browser '
 'extensions which could help protect the user from following potentially '
 

# Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.



In [34]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), min_len=2 ,deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

pprint(data_words[:1])

[['concerns',
  'regarding',
  'service',
  'authorization',
  'by',
  'ip',
  'address',
  'using',
  'eduroam',
  'eduroam',
  'is',
  'secure',
  'wlan',
  'roaming',
  'service',
  'between',
  'academic',
  'and',
  'research',
  'institutions',
  'around',
  'the',
  'globe',
  'it',
  'allows',
  'users',
  'from',
  'participating',
  'institutions',
  'secure',
  'internet',
  'access',
  'at',
  'any',
  'other',
  'participating',
  'visited',
  'institution',
  'using',
  'their',
  'home',
  'credentials',
  'the',
  'authentication',
  'credentials',
  'are',
  'verified',
  'by',
  'the',
  'home',
  'institution',
  'while',
  'authorization',
  'is',
  'done',
  'by',
  'the',
  'visited',
  'institution',
  'the',
  'user',
  'receives',
  'an',
  'ip',
  'address',
  'in',
  'the',
  'range',
  'of',
  'the',
  'visited',
  'institution',
  'and',
  'accesses',
  'the',
  'internet',
  'through',
  'the',
  'firewall',
  'and',
  'proxy',
  'servers',
  'of',
  'the'

### Building bigrams and trigrams

In [35]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [36]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['unfortunately','huge', 'used','may','using','jam','found','plays','however','information','security','good','aodv','would','derive','hence','focus','research','present','motive','study','result','within','propose','different','therefore','approach','form','also','new','context','df','pron','opportunity','institution','esc','process','author', 'work', 'base', 'easy','material','area','produce','period', 'publish','end','perform','less','become','seem','possible','vital','tool','be','make', 'move', 'space', 'uses','use','from', 'often','definition','ensure','much','useful','a','s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain','t','all','allow','allows','almost','alone','along','already','also,although,always,am,among,amongst,an,and,another,any,anybody,anyhow,anyone,anything,anyway,anyways,anywhere,apart,appear,appreciate,appropriate,are,aren,t,around,as,aside,ask,asking,associated,at,available,away,awfully,be,became,because,become,becomes,becoming,been,before,beforehand,behind,being,believe,below,beside,besides,best,better,between,beyond,both,brief,but,by,c,mon,c,s,came,can,can,t,cannot,cant,cause,causes,certain,certainly,changes,clearly,co,com,come,comes,concerning,consequently,consider,considering,contain,containing,contains,corresponding,could,couldn,t,course,currently,definitely,described,despite,did,didn,t,different,do,does,doesn,t,doing,don,t,done,down,downwards,during,each,edu,eg,eight,either,else,elsewhere,enough,entirely,especially,et,etc,even,ever,every,everybody,everyone,everything,everywhere,ex,exactly,example,except,far,few,fifth,first,five,followed,following,follows,for,former,formerly,forth,four,from,further,furthermore,get,gets,getting,given,gives,go,goes,going,gone,got,gotten,greetings,had,hadn,t,happens,hardly,has,hasn,t,have,haven,t,having,he,he,s,hello,help,hence,her,here,here,s,hereafter,hereby,herein,hereupon,hers,herself,hi,him,himself,his,hither,hopefully,how,howbeit,however,i,d,i,ll,i,m,i,ve,ie,if,ignored,immediate,in,inasmuch,inc,indeed,indicate,indicated,indicates,inner,insofar,instead,into,inward,is,isn,t,it,it,d,it,ll,it,s,its,itself,just,keep,keeps,kept,know,knows,known,last,lately,later,latter,latterly,least,less,lest,let,let,s,like,liked,likely,little,look,looking,looks,ltd,mainly,many,may,maybe,me,mean,meanwhile,merely,might,more,moreover,most,mostly,much,must,my,myself,name,namely,nd,near,nearly,necessary,need,needs,neither,never,nevertheless,new,next,nine,no,nobody,non,none,noone,nor,normally,not,nothing,novel,now,nowhere,obviously,of,off,often,oh,ok,okay,old,on,once,one,ones,only,onto,or,other,others,otherwise,ought,our,ours,ourselves,out,outside,over,overall,own,particular,particularly,per,perhaps,placed,please,plus,possible,presumably,probably,provides,que,quite,qv,rather,rd,re,really,reasonably,regarding,regardless,regards,relatively,respectively,right,said,same,saw,say,saying,says,second,secondly,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,shall,she,should,shouldn,t,since,six,so,some,somebody,somehow,someone,something,sometime,sometimes,somewhat,somewhere,soon,sorry,specified,specify,specifying,still,sub,such,sup,sure,t,s,take,taken,tell,tends,th,than,thank,thanks,thanx,that,that,s,thats,the,their,theirs,them,themselves,then,thence,there,there,s,thereafter,thereby,therefore,therein,theres,thereupon,these,they,they,d,they,ll,they,re,they,ve,think,third,this,thorough,thoroughly,those,though,three,through,throughout,thru,thus,to,together,too,took,toward,towards,tried,tries,truly,try,trying,twice,two,un,under,unfortunately,unless,unlikely,until,unto,up,upon,us,use,used,useful,uses,using,usually,value,various,very,via,viz,vs,want,wants,was,wasn,t,way,we,we,d,we,ll,we,re,we,ve,welcome,well,went,were,weren,t,what,what,s,whatever,when,whence,whenever,where,where,s,whereafter,whereas,whereby,wherein,whereupon,wherever,whether,which,while,whither,who,who,s,whoever,whole,whom,whose,why,will,willing,wish,with,within,without,won,t,wonder,would,would,wouldn,t,yes,yet,you,you,d,you,ll,you,re,you,ve,your,something,yours,yourself,yourselves,zero,the,of,and,to,is,that,was,for,it,in,om,it,i,it,that,is,in,and,of,to,a,the','subject', 're', 'edu','value', 'is', 'prahp', 'dfr', 'ecfr', 'help', 'current', 'number', 'show', 'find', 'ddo', 'day','site','paper'])
pprint(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [37]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



pprint(data_lemmatized[:1])

['concern regard service authorization ip address eduroam secure wlan roam '
 'service academic institution globe user participate institution secure '
 'internet access participate visit home credential authentication credential '
 'verify home authorization do visit user receive ip address range visit '
 'access internet firewall proxy server visit access grant service authorize '
 'ip address visit include access service allow home due legal agreement look '
 'typical legal agreement service provider explore risk countermeasure need '
 'consider eduroam']


In [38]:
# Building bigrams and trigrams
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops, min_count=2, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
pprint(trigram_mod[bigram_mod[data_words_nostops[0]]])




/Users/juandrevanheerden/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['concerns',
 'regarding',
 'service',
 'authorization',
 'ip_address',
 'eduroam',
 'eduroam',
 'secure',
 'wlan',
 'roaming',
 'service',
 'academic',
 'institutions',
 'around',
 'globe',
 'users',
 'participating',
 'institutions',
 'secure',
 'internet',
 'access',
 'participating',
 'visited',
 'home',
 'credentials',
 'authentication',
 'credentials',
 'verified',
 'home',
 'authorization',
 'done',
 'visited',
 'user',
 'receives',
 'ip_address',
 'range',
 'visited',
 'accesses',
 'internet',
 'firewall',
 'proxy',
 'servers',
 'visited',
 'access',
 'granted',
 'services',
 'authorize',
 'via',
 'ip_address',
 'visited',
 'include',
 'access',
 'services',
 'allowed',
 'home',
 'due',
 'legal',
 'agreements',
 'looks',
 'typical',
 'legal',
 'agreements',
 'service_providers',
 'explores',
 'risks',
 'countermeasures',
 'need',
 'considered',
 'eduroam']


In [39]:
# Creating dictionary, corpus, and TD-F
## Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

## Create Corpus
texts = data_words_nostops

## Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

## View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 3), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 1), (23, 2), (24, 2), (25, 3), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 3), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 5), (47, 1)]]


In [40]:
## Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('academic', 1),
  ('access', 3),
  ('accesses', 1),
  ('address', 3),
  ('agreements', 2),
  ('allowed', 1),
  ('around', 1),
  ('authentication', 1),
  ('authorization', 2),
  ('authorize', 1),
  ('concerns', 1),
  ('considered', 1),
  ('countermeasures', 1),
  ('credentials', 2),
  ('done', 1),
  ('due', 1),
  ('eduroam', 3),
  ('explores', 1),
  ('firewall', 1),
  ('globe', 1),
  ('granted', 1),
  ('home', 3),
  ('include', 1),
  ('institutions', 2),
  ('internet', 2),
  ('ip', 3),
  ('legal', 2),
  ('looks', 1),
  ('need', 1),
  ('participating', 2),
  ('providers', 1),
  ('proxy', 1),
  ('range', 1),
  ('receives', 1),
  ('regarding', 1),
  ('risks', 1),
  ('roaming', 1),
  ('secure', 2),
  ('servers', 1),
  ('service', 3),
  ('services', 2),
  ('typical', 1),
  ('user', 1),
  ('users', 1),
  ('verified', 1),
  ('via', 1),
  ('visited', 5),
  ('wlan', 1)],
 [('access', 1),
  ('due', 1),
  ('include', 1),
  ('acceptable', 1),
  ('acquire', 1),
  ('acquired', 1),
  ('addressing',

In [41]:
# Building LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=np.random.RandomState(0),
                                           iterations = 300,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [42]:
#import pickle
# Save
#lda_model.save("training.pkl")  # Stores *.pkl file
# Load
#lda = pickle.load("training.pkl")

In [43]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"mobile" + 0.025*"srtp" + 0.021*"secure" + 0.021*"implementation" + '
  '0.014*"magnusson" + 0.014*"voip" + 0.013*"technologies" + 0.012*"features" '
  '+ 0.012*"market" + 0.011*"subscribe"'),
 (1,
  '0.050*"risk" + 0.033*"management" + 0.019*"systems" + 0.019*"schools" + '
  '0.018*"risks" + 0.014*"cryptographic" + 0.014*"network" + 0.013*"networks" '
  '+ 0.012*"attacks" + 0.012*"computerised"'),
 (2,
  '0.038*"smes" + 0.029*"organisation" + 0.027*"role" + 0.020*"trust" + '
  '0.016*"considered" + 0.014*"usability" + 0.014*"effects" + '
  '0.013*"conceptual" + 0.012*"oisp" + 0.012*"play"'),
 (3,
  '0.022*"system" + 0.014*"based" + 0.011*"malicious" + 0.011*"users" + '
  '0.011*"analysis" + 0.010*"effectiveness" + 0.009*"behaviour" + '
  '0.008*"detection" + 0.008*"well" + 0.008*"impact"'),
 (4,
  '0.037*"social" + 0.030*"spam" + 0.027*"engineering" + 0.025*"warfare" + '
  '0.019*"techniques" + 0.017*"attack" + 0.017*"tetra" + 0.017*"email" + '
  '0.014*"voice" + 0.014*"

In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      11.712172        1       1  0.209813  0.010849
7      10.771890        1       2  0.185601 -0.033458
10      7.164501        1       3 -0.006004 -0.223587
15      6.663266        1       4  0.064920 -0.091845
17      6.306512        1       5  0.009235 -0.147389
19      5.938064        1       6  0.061131  0.016342
5       5.498569        1       7  0.095762  0.093107
9       5.327965        1       8  0.068788  0.068105
6       4.904278        1       9  0.080609  0.160085
16      4.564554        1      10  0.000128 -0.012921
1       3.979155        1      11  0.037400  0.109865
8       3.898364        1      12 -0.035143 -0.045042
18      3.332285        1      13 -0.090317  0.040679
14      3.241139        1      14 -0.031915  0.014592
13      3.119530        1      15 -0.194255  0.021694
12      3.109362        1      16 -0.168378  0.113417
11      3.020225        1      17 -0.044207 -0.096767
4       2.920418        1      18 -0.082785 -0.012676
0       2.420725        1      19 -0.078123  0.017869
2       2.107032        1      20 -0.082261 -0.002918, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
75    Default  189.000000         digital  189.000000  30.0000  30.0000
88    Default  136.000000        forensic  136.000000  29.0000  29.0000
70    Default  157.000000            data  157.000000  28.0000  28.0000
82    Default   77.000000        evidence   77.000000  27.0000  27.0000
565   Default   81.000000           south   81.000000  26.0000  26.0000
58    Default   78.000000           cloud   78.000000  25.0000  25.0000
936   Default   93.000000         privacy   93.000000  24.0000  24.0000
1248  Default   60.000000         android   60.000000  23.0000  23.0000
112   Default  100.000000         network  100.000000  22.0000  22.0000
39    Default   89.000000         service   89.000000  21.0000  21.0000
719   Default   57.000000            risk   57.000000  20.0000  20.0000
2050  Default   48.000000    architecture   48.000000  19.0000  19.0000
816   Default   81.000000       potential   81.000000  18.0000  18.0000
256   Default   56.000000          mobile   56.000000  17.0000  17.0000
702   Default   77.000000      management   77.000000  16.0000  16.0000
710   Default   45.000000    organisation   45.000000  15.0000  15.0000
1172  Default   45.000000          social   45.000000  14.0000  14.0000
728   Default   34.000000       standards   34.000000  13.0000  13.0000
296   Default   45.000000       awareness   45.000000  12.0000  12.0000
663   Default   44.000000         african   44.000000  11.0000  11.0000
40    Default   63.000000        services   63.000000  10.0000  10.0000
90    Default   49.000000       forensics   49.000000   9.0000   9.0000
272   Default   38.000000       readiness   38.000000   8.0000   8.0000
325   Default  105.000000       framework  105.000000   7.0000   7.0000
284   Default  108.000000          system  108.000000   6.0000   6.0000
97    Default   65.000000  implementation   65.000000   5.0000   5.0000
306   Default   39.000000         control   39.000000   4.0000   4.0000
513   Default   43.000000          africa   43.000000   3.0000   3.0000
1116  Default   43.000000             act   43.000000   2.0000   2.0000
285   Default   95.000000         systems   95.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
4539  Topic20    2.142168     obligations    2.921506   3.5496  -5.4959
4536  Topic20    2.142168           asked    2.921506   3.5496  -5.4959
4541  Topic20    2.142168        pointing    2.921506   3.5496  -5.4959
3780  Topic20    2.123503         economy    2.902825   3.5473  -5.5046
4269  Topic20    2.123502           brain    2.902826   3.5473  -5.5046
3959  Topic20    2.123500      ce

# Nuwe dokument

In [45]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/2016single.json')


print(df.abstracts.unique())
df.head()

['Team Formation in Digital Forensics A major problem in Digital Forensics (DF) is the often huge volumes of data that has to be searched, filtered, and indexed to discover patterns that could lead to forensic evidence. The nature of, and the process by which the data was collected also means that the data contain information about persons that are not involved, or only incidentally involved in the crime under investigation. Privacy is therefore another potential issue that needs to be dealt with in a DF investigation. This paper shows that techniques of the Team Formation (TF) task can be used to address both of these problems. The TF task can be formulated to fit the DF arena: to commit a crime, the culprit(s) may require the assistance of several other individuals, which implies that a team of some sort gets established. During a post-mortem DF analysis, an investigator may only have one, or a few names to start with. One of the key challenges is finding possible co-conspirators. Fr

,abstracts,title
0,Team Formation in Digital Forensics A major pr...,Team Formation in Digital Forensics


In [46]:
data1 = df.abstracts.values.tolist()




## Remove Emails
data1 = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

## Remove new line characters
data1 = [re.sub('\s+', ' ', sent) for sent in data]

## Remove fi ligature characters
data1 = [re.sub('ﬁ', 'fi', sent) for sent in data]

## Remove fi ligature characters
data1 = [re.sub('ﬂ', 'fl', sent) for sent in data]

## Remove single quotes
data1 = [re.sub("\'", "", sent) for sent in data]

pprint(data1[1:])

['Guidelines for Procedures of a Harmonised Digital Forensic Process in '
 'Network Forensics Cloud computing is a new computing paradigm that presents '
 'fresh research issues in the field of digital forensics. Cloud computing '
 'builds upon virtualisation technologies and is distributed in nature. '
 'Depending on its implementation, the cloud can span across numerous '
 'countries. Its distributed nature and virtualisation introduces digital '
 'forensic research issues that include among others difficulty in identifying '
 'and collecting forensically sound evidence. Even if the evidence may be '
 'identified and essential tools for collecting the evidence are acquired, it '
 'may be illegal to access computer data residing beyond the jurisdiction of a '
 'forensic investigator. The investigator needs to acquire a search warrant '
 'that can be executed in a specific foreign country – which may not be a '
 'single country due to the distributed nature of the cloud. Obtaining '
 '

 'facilitate regulatory compliance in the SA financial sector. A generic model '
 'is proposed, but elements from the abovementioned laws that address security '
 'and privacy, are used as a case study or proof-of-concept.',
 'A formal qualitative risk management approach for IT security Information '
 'technology (IT) security, which is concerned about protecting the '
 'confidentiality, integrity and availability of information technology '
 'assets, inherently possesses a significant amount of risk, some known and '
 'some unknown. IT security risk management has gained considerable attention '
 'over the past decade due to the collapsing of some large organisations in '
 'the world. Previous investigative research in the field of IT security have '
 'indicated that despite the efforts that organisations employ to reduce IT '
 'security risks, the trend of IT security attacks are still increasing. One '
 'of the contributing factors to poor management of IT security risk is '
 'attr

 'online, mobile, and Automated Teller Machine (ATM) banking. Digital banking, '
 'particularly online and mobile banking, is growing rapidly in South Africa. '
 'This is happening despite heightened security concerns, created in part by '
 'the media. The 2008 World Wide Worx and Wits Business School Digital Banking '
 'Research found that the level of sophistication related to the electronic '
 'related crime in the banking sector is extraordinary. The aforesaid research '
 'revealed that most of the digital banking crime affect online banking and '
 'ATM. The media captured the crime phenomenon by using catchy headlines. The '
 '2008 World Wide Worx and Wits Business School Digital Banking Research '
 'employed both qualitative research approaches. Quantitative data collection '
 'method was employed at an elementary level to capture relevant statistics. '
 'Purposive sampling was used to select the participating banks. Researchers '
 'deliberately targeted the biggest banks in Sout

 'is whether different thresholds apply to the purposes in conducting state '
 'electronic communications’ surveillance. The aim of the discussion is not to '
 'vilify government surveillance practices as governments carry the onerous '
 'task of protecting nationals against threats within the ever-advancing '
 'electronic communication technology medium. The aim is rather to establish '
 'whether there exist voids and/or deficiencies and if affirmative, how it '
 'could be addressed to ensure an open and transparent surveillance landscape '
 'aimed at striking a balance between security and human rights protection',
 'Challenges to Digital Forensics A Survey of Researchers & Practitioners '
 'Attitudes and Opinions Digital forensics have become increasingly important '
 'as an approach to investigate cyber- and computerassisted crime. Whilst many '
 'tools exist and much research is being undertaken, many questions exist '
 'regarding the future of the domain. Indeed, prior literature

 'enhance the company’s reactive capability to security incidents. The aim of '
 'this paper is to propose proactive activities an organisation can undertake '
 'in order to increase its ability to respond to security incidents and create '
 'a digitally forensic ready workplace environment. The study constitutes '
 'exploratory research, with the use of a systematic literature review as a '
 'basis to identify activities relating to a digitally forensic ready '
 'environment. While much has been written about how organisations can prepare '
 'to respond to security incidents, findings show an absence of a digital '
 'forensic readiness model. This paper concludes by presenting such a '
 'conceptual model. This study contributes to the greater body of knowledge on '
 'the design and implementation of a digital forensic readiness programme, '
 'aimed at maximising the use of digital evidence in an organisation.',
 'Information security governance control through comprehensive policy '
 

 'Internet using an unprotected system. The potential impact can be '
 'irreparable harm to a computer file system or even place a person in a '
 'situation where they could be charged for a criminal act, if the perpetrator '
 'assumes control of their system. Understanding contemporary forms of malware '
 'is crucial in order to prepare better defences against it as well as '
 'investigate related incidents and claims. Therefore forensic analysis of '
 'specific malware, requires specialised tools and techniques and is of '
 'significant importance for information security professionals.In an effort '
 'to facilitate the process of forensic analysis of malicious and hostile code '
 'we intend to develop a system whereby specific malware can be identified, '
 'classified and the malware and detailed forensic analysis stored in a '
 'searchable database. The research results would assist computer forensics '
 'expert witnesses and infosecurity specialists, to determine the potential '
 

 'not necessarily offer the same features. Decoupling of user identity from '
 'the main DRM system also reduces the chances of correlating users’ access '
 'patterns of protected works. This improves the privacy of users of DRM '
 'systems, another major criticisms of current DRM systems. In this paper we '
 'discuss the requirements for user identity in a DRM system and then '
 'introduce a Kerberos like reusable ticket based user identity system. This '
 'system allows multiple systems to be authenticated by the use of time '
 'limited authentication tickets, without requiring online authentication. '
 'Tickets can be stored at a central controlling point, which is also '
 'responsible for acquiring tickets from authentication servers and '
 'redistributing tickets to the devices that need the tickets. In our '
 'experience, our approach fulfils all the requirements and is a more scalable '
 'and inter-operable approach when compared to existing DRM systems',
 'The Use of Web Beacon

 'regular use of an installation is identifiable through entropy fingerprint '
 'analysis. The results obtained allow the rapid identification of several '
 'volume-level operations including copying and wiping, but also to detect '
 'anomalous slack space entropy indicative of the use of encryption '
 'techniques. Similarly, entropy and randomness tests have been devised which '
 'provide heuristics for the differentiation of encrypted data from other '
 'high-entropy data such as compressed media data.',
 'SPAM OVER INTERNET TELEPHONY AND HOW TO DEAL WITH IT In our modern society '
 'telephony has developed to an omnipresent service. People are available at '
 'anytime and anywhere. Furthermore the Internet has emerged to an important '
 'communication medium. These facts and the raising availability of broadband '
 'internet access has led to the fusion of these two services. Voice over IP '
 'or short VoIP is the keyword, that describes this combination. The '
 'advantages of VoIP 

 'A Conceptual Opportunity-based Framework to Mitigate the Insider Threat The '
 'aim of this paper is to provide a conceptual framework to mitigate the '
 'insider threat from an opportunitybased perspective. Although motive and '
 'opportunity are required to commit maleficence, this paper focuses on the '
 'concept of opportunity. Opportunity is more tangible than motive, hence it '
 'is more pragmatic to reflect on opportunity-reducing measures. Opportunity '
 'theories from the field of criminology are considered to this end. The '
 'derived framework highlights several areas of research and may assist '
 'organisations in designing controls that are situationally appropriate to '
 'mitigate insider threat. Current information security countermeasures are '
 'not designed from an opportunity-reducing perspective.']


In [47]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), min_len=2 ,deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data1))

pprint(data_words[:1])

[['concerns',
  'regarding',
  'service',
  'authorization',
  'by',
  'ip',
  'address',
  'using',
  'eduroam',
  'eduroam',
  'is',
  'secure',
  'wlan',
  'roaming',
  'service',
  'between',
  'academic',
  'and',
  'research',
  'institutions',
  'around',
  'the',
  'globe',
  'it',
  'allows',
  'users',
  'from',
  'participating',
  'institutions',
  'secure',
  'internet',
  'access',
  'at',
  'any',
  'other',
  'participating',
  'visited',
  'institution',
  'using',
  'their',
  'home',
  'credentials',
  'the',
  'authentication',
  'credentials',
  'are',
  'verified',
  'by',
  'the',
  'home',
  'institution',
  'while',
  'authorization',
  'is',
  'done',
  'by',
  'the',
  'visited',
  'institution',
  'the',
  'user',
  'receives',
  'an',
  'ip',
  'address',
  'in',
  'the',
  'range',
  'of',
  'the',
  'visited',
  'institution',
  'and',
  'accesses',
  'the',
  'internet',
  'through',
  'the',
  'firewall',
  'and',
  'proxy',
  'servers',
  'of',
  'the'

In [48]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [49]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['huge','unfortunately','used','may','using','jam','found','plays','however','information','security','good','aodv','would','derive','hence','focus','research','present','motive','study','result','within','propose','different','therefore','approach','form','also','new','context','df','pron','opportunity','institution','esc','process','author', 'work', 'base', 'easy','material','area','produce','period', 'publish','end','perform','less','become','seem','possible','vital','tool','be','make', 'move', 'space', 'uses','use','from', 'often','definition','ensure','much','useful','a','s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain','t','all','allow','allows','almost','alone','along','already','also,although,always,am,among,amongst,an,and,another,any,anybody,anyhow,anyone,anything,anyway,anyways,anywhere,apart,appear,appreciate,appropriate,are,aren,t,around,as,aside,ask,asking,associated,at,available,away,awfully,be,became,because,become,becomes,becoming,been,before,beforehand,behind,being,believe,below,beside,besides,best,better,between,beyond,both,brief,but,by,c,mon,c,s,came,can,can,t,cannot,cant,cause,causes,certain,certainly,changes,clearly,co,com,come,comes,concerning,consequently,consider,considering,contain,containing,contains,corresponding,could,couldn,t,course,currently,definitely,described,despite,did,didn,t,different,do,does,doesn,t,doing,don,t,done,down,downwards,during,each,edu,eg,eight,either,else,elsewhere,enough,entirely,especially,et,etc,even,ever,every,everybody,everyone,everything,everywhere,ex,exactly,example,except,far,few,fifth,first,five,followed,following,follows,for,former,formerly,forth,four,from,further,furthermore,get,gets,getting,given,gives,go,goes,going,gone,got,gotten,greetings,had,hadn,t,happens,hardly,has,hasn,t,have,haven,t,having,he,he,s,hello,help,hence,her,here,here,s,hereafter,hereby,herein,hereupon,hers,herself,hi,him,himself,his,hither,hopefully,how,howbeit,however,i,d,i,ll,i,m,i,ve,ie,if,ignored,immediate,in,inasmuch,inc,indeed,indicate,indicated,indicates,inner,insofar,instead,into,inward,is,isn,t,it,it,d,it,ll,it,s,its,itself,just,keep,keeps,kept,know,knows,known,last,lately,later,latter,latterly,least,less,lest,let,let,s,like,liked,likely,little,look,looking,looks,ltd,mainly,many,may,maybe,me,mean,meanwhile,merely,might,more,moreover,most,mostly,much,must,my,myself,name,namely,nd,near,nearly,necessary,need,needs,neither,never,nevertheless,new,next,nine,no,nobody,non,none,noone,nor,normally,not,nothing,novel,now,nowhere,obviously,of,off,often,oh,ok,okay,old,on,once,one,ones,only,onto,or,other,others,otherwise,ought,our,ours,ourselves,out,outside,over,overall,own,particular,particularly,per,perhaps,placed,please,plus,possible,presumably,probably,provides,que,quite,qv,rather,rd,re,really,reasonably,regarding,regardless,regards,relatively,respectively,right,said,same,saw,say,saying,says,second,secondly,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,shall,she,should,shouldn,t,since,six,so,some,somebody,somehow,someone,something,sometime,sometimes,somewhat,somewhere,soon,sorry,specified,specify,specifying,still,sub,such,sup,sure,t,s,take,taken,tell,tends,th,than,thank,thanks,thanx,that,that,s,thats,the,their,theirs,them,themselves,then,thence,there,there,s,thereafter,thereby,therefore,therein,theres,thereupon,these,they,they,d,they,ll,they,re,they,ve,think,third,this,thorough,thoroughly,those,though,three,through,throughout,thru,thus,to,together,too,took,toward,towards,tried,tries,truly,try,trying,twice,two,un,under,unfortunately,unless,unlikely,until,unto,up,upon,us,use,used,useful,uses,using,usually,value,various,very,via,viz,vs,want,wants,was,wasn,t,way,we,we,d,we,ll,we,re,we,ve,welcome,well,went,were,weren,t,what,what,s,whatever,when,whence,whenever,where,where,s,whereafter,whereas,whereby,wherein,whereupon,wherever,whether,which,while,whither,who,who,s,whoever,whole,whom,whose,why,will,willing,wish,with,within,without,won,t,wonder,would,would,wouldn,t,yes,yet,you,you,d,you,ll,you,re,you,ve,your,something,yours,yourself,yourselves,zero,the,of,and,to,is,that,was,for,it,in,om,it,i,it,that,is,in,and,of,to,a,the','subject', 're', 'edu','value', 'is', 'prahp', 'dfr', 'ecfr', 'help', 'current', 'number', 'show', 'find', 'ddo', 'day','site','paper'])
pprint(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [50]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



pprint(data_lemmatized[:1])

['concern regard service authorization ip address eduroam secure wlan roam '
 'service academic institution globe user participate institution secure '
 'internet access participate visit home credential authentication credential '
 'verify home authorization do visit user receive ip address range visit '
 'access internet firewall proxy server visit access grant service authorize '
 'ip address visit include access service allow home due legal agreement look '
 'typical legal agreement service provider explore risk countermeasure need '
 'consider eduroam']


In [51]:
# Building bigrams and trigrams
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_nostops, min_count=1, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
pprint(trigram_mod[bigram_mod[data_words_nostops[0]]])




/Users/juandrevanheerden/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['concerns_regarding',
 'service',
 'authorization',
 'ip_address',
 'eduroam',
 'eduroam',
 'secure',
 'wlan',
 'roaming',
 'service',
 'academic',
 'institutions',
 'around',
 'globe',
 'users',
 'participating',
 'institutions',
 'secure',
 'internet_access',
 'participating',
 'visited',
 'home',
 'credentials',
 'authentication',
 'credentials',
 'verified',
 'home',
 'authorization',
 'done',
 'visited',
 'user',
 'receives',
 'ip_address',
 'range',
 'visited',
 'accesses',
 'internet',
 'firewall',
 'proxy',
 'servers',
 'visited',
 'access_granted',
 'services',
 'authorize',
 'via',
 'ip_address',
 'visited',
 'include',
 'access',
 'services',
 'allowed',
 'home',
 'due',
 'legal_agreements',
 'looks',
 'typical',
 'legal_agreements',
 'service_providers',
 'explores',
 'risks',
 'countermeasures',
 'need_considered',
 'eduroam']


In [71]:
# Creating dictionary, corpus, and TD-F
## Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

## Create Corpus
texts = data_words_nostops

## Term Document Frequency
corpus1 = [id2word.doc2bow(text) for text in texts]

## View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 3), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 1), (23, 2), (24, 2), (25, 3), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 3), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 5), (47, 1)]]


In [72]:
## Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('academic', 1),
  ('access', 3),
  ('accesses', 1),
  ('address', 3),
  ('agreements', 2),
  ('allowed', 1),
  ('around', 1),
  ('authentication', 1),
  ('authorization', 2),
  ('authorize', 1),
  ('concerns', 1),
  ('considered', 1),
  ('countermeasures', 1),
  ('credentials', 2),
  ('done', 1),
  ('due', 1),
  ('eduroam', 3),
  ('explores', 1),
  ('firewall', 1),
  ('globe', 1),
  ('granted', 1),
  ('home', 3),
  ('include', 1),
  ('institutions', 2),
  ('internet', 2),
  ('ip', 3),
  ('legal', 2),
  ('looks', 1),
  ('need', 1),
  ('participating', 2),
  ('providers', 1),
  ('proxy', 1),
  ('range', 1),
  ('receives', 1),
  ('regarding', 1),
  ('risks', 1),
  ('roaming', 1),
  ('secure', 2),
  ('servers', 1),
  ('service', 3),
  ('services', 2),
  ('typical', 1),
  ('user', 1),
  ('users', 1),
  ('verified', 1),
  ('via', 1),
  ('visited', 5),
  ('wlan', 1)],
 [('access', 1),
  ('due', 1),
  ('include', 1),
  ('acceptable', 1),
  ('acquire', 1),
  ('acquired', 1),
  ('addressing',

In [75]:
# Building LDA model
lda1_model = gensim.models.ldamodel.LdaModel(corpus=corpus1,
                                           id2word=id2word,
                                           num_topics=1,
                                           random_state=np.random.RandomState(0),
                                           iterations = 300,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [76]:
pprint(lda1_model.print_topics())

[(0,
  '0.007*"digital" + 0.007*"data" + 0.005*"forensic" + 0.004*"based" + '
  '0.004*"system" + 0.004*"service" + 0.004*"framework" + 0.004*"network" + '
  '0.004*"privacy" + 0.004*"systems"')]


In [77]:
doc_topic_dist = np.array([[tup for tup in lst] for lst in lda_model[corpus]])
doc_topic_dist.shape

(235,)

In [78]:
def jensen_shannon(query, matrix):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    # lets keep with the p,q notation above
    p = query[None,:].T # take transpose
    q = matrix.T # transpose matrix
    m = 0.5*(p + q)
    return np.sqrt(0.5*(entropy(p,m) + entropy(q,m)))

In [79]:
def get_most_similar_documents(query,matrix,k=10):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    sims = jensen_shannon(query,matrix) # list of jensen shannon distances
    return sims.argsort()[:k] # the top k positional index of the smallest Jensen Shannon distances

In [80]:
new_doc_distribution = np.array([tup for tup in lda_model.get_document_topics(bow=corpus1)])

In [81]:
# this is surprisingly fast
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)

TypeError: can't multiply sequence by non-int of type 'float'